In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
df = spark.read.csv(
    "S32W.tsv", sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0007,high tech bite of equipment
S0018,it's very professional
S0007,mm
S0018,it give really good sound quality as well fantastic so yeah so you use to be a sport journalist
S0007,well yeah i'm still try to so i be in the loose sense of the word um i er go to university six years ago now and um yeah like er i never grow up want to be a sport journalist i want to be a a sport athlete or sport performer


In [3]:
def extract_speakers(df1):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0]!= speaker[0]:
            speaker.append(row[0])
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [4]:
def extract_dialogues_dic(rdd1, speaker):
    speaker1 = speaker[0]
    speaker2 = speaker[1]
    speaker1_dialogue = {}
    speaker2_dialogue = {}
    dataC = rdd1.collect()
    for i in range(0,len(dataC)):
        if dataC[i][0] == speaker1:
            if dataC[i][1]!=None:
                speaker1_dialogue[i] = dataC[i][1].split()
#             speaker1_dialogue.append(dataC[i][1].split())
        elif dataC[i][0] == speaker2:
            if dataC[i][1]!=None:
                speaker2_dialogue[i] = dataC[i][1].split()
#             speaker2_dialogue.append(dataC[i][1].split())
    
    return speaker, speaker1_dialogue, speaker2_dialogue

In [5]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(stops)

{'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', "hasn't", 'then', 'both', 'so', 're', 'don', 'm', 'as', 'any', 'mightn', 'after', 'you', 'wouldn', 'why', 'been', 'where', 'by', "isn't", 'yourself', 'wasn', 'a', "haven't", 'did', "hadn't", 'their', 'hasn', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', "you'll", 'yourselves', 'that', 'my', 'what', 'to', 'd', 'not', "won't", "couldn't", 'own', 'there', 'this', 'each', 'all', 'haven', 'more', 'me', 've', 'weren', 'which', 'himself', 'nor', 'other', "shouldn't", 'who', "should've", 'same', 'at', 'such', 't', 'up', 'than', 'can', "you've", 'too', 'these', 'while', "wasn't", 'ourselves', 'before', 'i', 'he', "didn't", 'our', 'its', 'but', 'with', "wouldn't", 'those', 'because', 'the', 'y', 'shouldn', 'it', 'mustn', 'hers', 'just', 'doesn', 'ain', 'between', 'over', 'had', 'aren', "mightn't", 'does', 'have', 'and', 'or', 'some', "mustn't", 'only', 'won', 'when', 'needn', 'below', 'in', 'if', 'theirs', "needn't", "aren't",

In [6]:
def get_er_one_with_u(x,y, exclude_stops):
# need to be changed in other files
    record=[[0 for i in range(len(y)+1)]  for j in range(len(x)+1)]
    dic_record_s1 = {}
    dic_record_s2 = {}
    expression_list_1 = []
    expression_list_2 = []
    expression_list_1_token = []
    expression_list_2_token = []
    for i in range(0,len(x)):
        for j in range(0,len(y)):
            if x[i] == y[j]:
                record[i+1][j+1] = record[i][j] + 1
                if record[i+1][j+1] > 0:
                        if i not in dic_record_s1.keys():
                            dic_record_s1[i] = record[i+1][j+1]
                        elif i in dic_record_s1.keys():
                            if (dic_record_s1[i]<record[i+1][j+1]):
                                dic_record_s1[i] = record[i+1][j+1]
                                
                        if j not in dic_record_s2.keys():
                            dic_record_s2[j] = record[i+1][j+1]
                        elif j in dic_record_s2.keys():
                            if (dic_record_s2[j]<record[i+1][j+1]):
                                dic_record_s2[j] = record[i+1][j+1]

    if dic_record_s1== {}:
        return []
    
    else:
        i = max(dic_record_s1.values())
        
        
    if dic_record_s2== {}:
        return []
    
    else:
        j = max(dic_record_s2.values())
    
    while i>0:  
        for key in dic_record_s1:
            if dic_record_s1[key] == i:
            #expression_repitition_record = expression_repitition_record+1
                expression = x[key+1-i : key+1]
                expression_string = ' '.join(expression)
            
                for k in range(0,i):
                    dic_record_s1[key - k] = 0
            
#             if expression_list != []:
#                 for e in expression_list:
#                     if expression not in e:
#                         expression_list.append(expression) 
#             else:

#                 if expression_list == []:
#                     expression_list.append(expression_string)
#                 else:
#                     for ex in expression_list:
#                         if expression string not in ex:
#                             expression_list.append(expression)
                if exclude_stops == True:
                    if expression_string not in stops:
                        if expression_string not in expression_list_1:
                            expression_list_1.append(expression_string)
                            expression_list_1_token.append(expression)
                                     
                else:
                    if expression_string not in expression_list_1:
                        expression_list_1.append(expression_string)
#                 lens1 = lens1 + len(expression)
                        expression_list_1_token.append(expression)
        
        i = i-1
        
    while j>0:  
        for key in dic_record_s2:
            if dic_record_s2[key] == j:
            #expression_repitition_record = expression_repitition_record+1
                expression_2 = y[key+1-j : key+1]
                expression_string_2 = ' '.join(expression_2)
            
                for k in range(0,j):
                    dic_record_s2[key - k] = 0
                
                
                
                if exclude_stops == True:
                    if expression_string_2 not in stops:
                        if expression_string_2 not in expression_list_1:
                            expression_list_1.append(expression_string_2)
                            expression_list_1_token.append(expression_2)
                
                else:
                    if expression_string_2 not in expression_list_1:
                        expression_list_1.append(expression_string_2)
                        expression_list_1_token.append(expression_2)
        
        j = j-1
        
    return expression_list_1

In [7]:
def get_er_dic_in_window(speaker, speaker1_dialogue, speaker2_dialogue,u_low, u_high):
    expressions = {}
    
    for x,value_x in speaker1_dialogue.items():
        for y,value_y in speaker2_dialogue.items():
            if (x<u_high) and (x>= u_low) and (y<u_high) and (y>= u_low):
                string = get_er_one_with_u(value_x, value_y, False)
                value_x_sentence = ' '.join(value_x)
                value_y_sentence = ' '.join(value_y)
            
                for s in string:
                
                    if s not in expressions.keys():
                    
                        pos = max(x,y)
                        expressions[s] = [pos,[x],[y]]
                    
                    elif s in expressions.keys():
                        if (x in expressions[s][1]) and (y not in expressions[s][2]):
                            expressions[s][2].append(y)
                            expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
                        elif (y in expressions[s][2]) and (x not in expressions[s][1]):
                            expressions[s][1].append(x)
                            expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
#                     elif (y in expressions[s][2]) and (x in expressions[s][1]):
#                         expressions[s][0] = expressions[s][0]+count1+count2
                    
#                     expressions[s][1].append(x)
#                     expressions[s][2].append(y)
    
    return expressions

In [8]:
# def ex_in_timeslot(expression_dic,u_slot):
#     ex_in_time = {}
#     for k, v in expression_dic.items():
#         flag = True
#         for x in v[1]:
#             for y in v[2]:
#                 if (v[0] <= u_slot) and (x < u_slot) and (y < u_slot):
#                     if k not in ex_in_time.keys():
#                         ex_in_time[k] = v
#                         flag = False
#                         break
#             if not flag:
#                 break
#     return ex_in_time  

In [9]:
def get_er_ratio_in_time_before(speaker, speaker1_dialogue, speaker2_dialogue, ex_in_u, u_low, u_high):
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high) and (u1>= value[0]):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                        c1 = dialogue_s1[u1].count(key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
                        count_u1 = c1+count_u1
                        count_u1_token = i*c1 + count_u1_token
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high) and (u2>= value[0]):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                        c2 = dialogue_s2[u2].count(key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        count_u2 = c2+count_u2
                        count_u2_token = i*c2 + count_u2_token
    
        i = i-1
        
    for key,value in ex_in_u.items():
        if len(key.split()) == 1:
            for u1 in value[1]:
                if (u1>=u_low) and (u1<u_high) and (u1>= value[0]):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                    c1 = dialogue_s1[u1].split().count(key)
    
#                     dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
#                 print(dialogue_dic_s1[u1])
                    count_u1 = c1+count_u1
                    count_u1_token = i*c1 + count_u1_token
            for u2 in value[2]:
                if (u2>=u_low) and (u2<u_high) and (u2>= value[0]):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                    c2 = dialogue_s2[u2].split().count(key)
#                     dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                    count_u2 = c2+count_u2
                    count_u2_token = i*c2 + count_u2_token
    
             
        
    if token_sum_s1 != 0:
        speaker1_ratio = count_u1_token/token_sum_s1
    else:
        speaker1_ratio = 0
    if token_sum_s2 != 0:
        speaker2_ratio = count_u2_token/token_sum_s2
    else:
        speaker2_ratio = 0
            
    if (token_sum_s2+token_sum_s1) != 0:
        er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
    else:
        er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [10]:
import copy
import pandas as pd
def get_er_ratio_in_time(speaker, speaker1_dialogue, speaker2_dialogue, u_low, u_high):
    ex_in_u = get_er_dic_in_window(speaker, speaker1_dialogue, speaker2_dialogue,u_low, u_high)
    
    
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
#     printout =[]
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high):
                        c1 = dialogue_s1[u1].count(key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
                        if (u1>= value[0]):                        
                            count_u1 = c1+count_u1
                            count_u1_token = len(key.split())*c1 + count_u1_token
#                             dic_out = {'key':key, "u":u1, "count_token":c1, "speaker":speaker[0]}
#                             printout.append(dic_out)
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high):
                        c2 = dialogue_s2[u2].count(key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        if (u2>= value[0]):
                            count_u2 = c2+count_u2
                            count_u2_token = len(key.split())*c2 + count_u2_token
#                             dic_out = {'key':key, "u":u2, "count_token":c2, "speaker":speaker[1]}
#                             printout.append(dic_out)
    
        i = i-1
    
    for key,value in ex_in_u.items():
            if len(key.split()) == 1:
                for u1 in value[1]:
                    if (u1>=u_low) and (u1<u_high):
                        c1 = dialogue_s1[u1].split().count(key)
                        if (u1>= value[0]):
                            count_u1 = c1+count_u1
                            count_u1_token = c1 + count_u1_token
#                             dic_out = {'key':key, "u":u1, "count_token":c1, "speaker":speaker[0]}
#                             printout.append(dic_out)

                        
                for u2 in value[2]:
                    if (u2>=u_low) and (u2<u_high) :
                        c2 = dialogue_s2[u2].split().count(key)
                        if (u2>= value[0]):
                            count_u2 = c2+count_u2
                            count_u2_token = c2 + count_u2_token
#                             dic_out = {'key':key, "u":u2, "count_token":c2, "speaker":speaker[1]}
#                             printout.append(dic_out)
    
#     dl_test = pd.DataFrame(printout)
#     dl_test.to_csv(('test_result_s2a5.csv'), index = False)
    
        
    if token_sum_s1 != 0:
        speaker1_ratio = count_u1_token/token_sum_s1
    else:
        speaker1_ratio = 0
    if token_sum_s2 != 0:
        speaker2_ratio = count_u2_token/token_sum_s2
    else:
        speaker2_ratio = 0
            
    if (token_sum_s2+token_sum_s1) != 0:
        er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
    else:
        er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [11]:
def get_er_ratio_in_time_after(speaker, speaker1_dialogue, speaker2_dialogue, ex_in_u, u_low, u_high):
    max_len = 0
    for key in ex_in_u.keys():
        max_len = max(len(key.split()),max_len)
    
    i = max_len
    dialogue_s1 = {}
    dialogue_s2 = {}
    count_u1 = 0
    count_u2 = 0
    count_u1_token = 0
    count_u2_token = 0
    token_sum_s1 = 0
    token_sum_s2 = 0
    
    for k,v in speaker1_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s1 = token_sum_s1 + len(v)
            dialogue_s1[k] = ' '.join(speaker1_dialogue[k])
            
    
    for k,v in speaker2_dialogue.items():
        if k<u_high and k>=u_low:
            token_sum_s2 = token_sum_s2 + len(v)
            dialogue_s2[k] = ' '.join(speaker2_dialogue[k])
        
    while i>1:
        for key,value in ex_in_u.items():
            if len(key.split()) == i:
                for u1 in value[1]:
                    if (u1>=u_low and u1<u_high):
#                         print(u1,key)
#                 dialogue_dic_s1[u1] = ' '.join(dialogue_dic_s1[u1])
                        c1 = dialogue_s1[u1].count(' ' + key)
                        dialogue_s1[u1] = dialogue_s1[u1].replace(key, '0'*len(key))
#                 print(dialogue_dic_s1[u1])
                        count_u1 = c1+count_u1
                        count_u1_token = i*c1 + count_u1_token
                for u2 in value[2]:
                    if (u2>=u_low and u2<u_high):
#                         print(u2,key)
#                 dialogue_dic_s2[u2] = ' '.join(dialogue_dic_s2[u2])
                        c2 = dialogue_s2[u2].count(' ' + key)
                        dialogue_s2[u2] = dialogue_s2[u2].replace(key, '0'*len(key))
                        count_u2 = c2+count_u2
                        count_u2_token = i*c2 + count_u2_token
    
        i = i-1
        
      
        if token_sum_s1 != 0:
            speaker1_ratio = count_u1_token/token_sum_s1
        else:
            speaker1_ratio = 0
    
        if token_sum_s2 != 0:
            speaker2_ratio = count_u2_token/token_sum_s2
        else:
            speaker2_ratio = 0
            
        if (token_sum_s2+token_sum_s1) != 0:
            er_ratio = (count_u1_token+count_u2_token)/(token_sum_s1+token_sum_s2)
        else:
            er_ratio = 0
    
    return speaker1_ratio,speaker2_ratio,er_ratio

In [12]:
def number_of_slot(dialogue1,dialogue2,length_u):
    number_of_u = len(dialogue1)+len(dialogue2)
    number_slot = int(number_of_u - number_of_u%length_u)/length_u

    return int(number_slot)

In [22]:
def get_er_ratio_temporal(speaker, speaker1_dialogue, speaker2_dialogue, length):
    slot_number = number_of_slot(speaker1_dialogue, speaker2_dialogue,length)
    er_dic = {}
    er_dic_list = []
    
    for i in range(0,slot_number):
        er = get_er_ratio_in_time(speaker,speaker1_dialogue,speaker2_dialogue,length*i,length*(i+1))
        er_dic = {'begin':length*i, 'end': length*(i+1), 's1_er_ratio':er[0],'s2_er_ratio':er[1],'er_ratio_average':er[2]}
        er_dic_list.append(er_dic)
    
    er_end = get_er_ratio_in_time(speaker,speaker1_dialogue,speaker2_dialogue,slot_number*length,len(speaker1_dialogue)+len(speaker2_dialogue))
    er_dic_end = {'begin':slot_number*length, 'end': len(speaker1_dialogue)+len(speaker2_dialogue), 's1_er_ratio':er_end[0], 's2_er_ratio':er_end[1], 'er_ratio_average':er_end[2]}  
    er_dic_list.append(er_dic_end)
    
    return er_dic_list
        

In [14]:
speaker = extract_speakers(df)
dialogue_dic = extract_dialogues_dic(speaker[0],speaker[1])

In [15]:
ex_dic = get_er_dic_in_window(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],0,50)

In [16]:
print(ex_dic)

{'mm': [11, [2], [11, 39, 43]], 'to be a sport journalist': [4, [4], [3]], 'to be a': [4, [4], [3]], 'a sport': [4, [4], [3]], 'well': [4, [4, 10, 12, 13], [3]], 'yeah': [4, [4, 15, 16, 20, 34, 36], [3, 5, 21, 33, 37]], 'to': [4, [4, 10, 15, 26, 40, 42], [3, 33, 35]], 'so': [4, [4, 16, 24, 30, 32, 42, 46], [3]], 'be': [4, [4, 6, 8, 10, 13, 16, 20, 26, 30, 32, 40, 48], [3, 19, 33]], 'sport': [4, [4, 24, 26], [3, 5, 19]], 'sport journalist': [19, [4], [19]], 'a a': [19, [4, 48], [19]], 'a': [19, [4, 18, 20, 22, 24, 26, 28, 30, 36, 40, 42, 44, 48], [19]], 'um yeah': [33, [4], [33]], 'i': [33, [4, 8, 10, 12, 13, 16, 26, 40, 42, 44, 46, 48], [33]], 'in': [33, [4, 13, 26, 30, 42], [33]], 'the': [33, [4, 6, 10, 13, 16, 20, 22, 26, 30, 32, 40, 42, 44, 48], [33, 35]], 'um': [33, [4, 10, 13, 16, 18, 26, 42], [33]], 'years': [33, [4, 18, 44], [33]], 'like': [33, [4, 10, 13, 15, 44], [33]], 'two': [33, [8, 44], [33]], 'it': [10, [10, 13, 16, 20, 26, 38, 40, 42], [3, 33]], 'what': [10, [10, 12], [5

In [17]:
print(len(ex_dic))

52


In [18]:
er_list_2 = get_er_ratio_in_time(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],0 ,50)

In [19]:
for x in er_list_2:
    print(x)

0.30095238095238097
0.3958333333333333
0.31561996779388085


In [20]:
slots = number_of_slot(dialogue_dic[1],dialogue_dic[2],50)

In [21]:
print(slots)

23


In [24]:
er_result = get_er_ratio_temporal(dialogue_dic[0],dialogue_dic[1],dialogue_dic[2],50)

In [26]:
for e in er_result:
    print(e['er_ratio_average'])

0.31561996779388085
0.18615384615384614
0.3217391304347826
0.381201044386423
0.19
0.11145510835913312
0.24306688417618272
0.26009693053311794
0.406807131280389
0.3632286995515695
0.4350282485875706
0.4409221902017291
0.42685851318944845
0.2420814479638009
0.36597938144329895
0.3129161118508655
0.22408026755852842
0.3277777777777778
0.38571428571428573
0.38768115942028986
0.3974630021141649
0.2620689655172414
0.3226452905811623
0.3664302600472813


In [27]:
print(er_result)

[{'begin': 0, 'end': 50, 's1_er_ratio': 0.30095238095238097, 's2_er_ratio': 0.3958333333333333, 'er_ratio_average': 0.31561996779388085}, {'begin': 50, 'end': 100, 's1_er_ratio': 0.4472361809045226, 's2_er_ratio': 0.6226415094339622, 'er_ratio_average': 0.46153846153846156}, {'begin': 100, 'end': 150, 's1_er_ratio': 0.5726141078838174, 's2_er_ratio': 0.6774193548387096, 'er_ratio_average': 0.5895652173913043}, {'begin': 150, 'end': 200, 's1_er_ratio': 0.6329113924050633, 's2_er_ratio': 0.664179104477612, 'er_ratio_average': 0.6383812010443864}, {'begin': 200, 'end': 250, 's1_er_ratio': 0.641213389121339, 's2_er_ratio': 0.5681818181818182, 'er_ratio_average': 0.638}, {'begin': 250, 'end': 300, 's1_er_ratio': 0.6767317939609236, 's2_er_ratio': 0.7228915662650602, 'er_ratio_average': 0.6826625386996904}, {'begin': 300, 'end': 350, 's1_er_ratio': 0.6490566037735849, 's2_er_ratio': 0.7590361445783133, 'er_ratio_average': 0.6639477977161501}, {'begin': 350, 'end': 400, 's1_er_ratio': 0.69675

In [27]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles/output_er_summary_dialign_in_window'
dl = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers(dff)
    dialogue_dic_f = extract_dialogues_dic(speaker_f[0],speaker_f[1])
#     ex_dic_f = get_er_dic(dialogue_dic_f[0],dialogue_dic_f[1],dialogue_dic_f[2])
#     for k,v in ex_dic_f.items():
#         print(f,k,',',v)
#     print(dialogue_dic_f[1][4])
    er_result_f = get_er_ratio_temporal(dialogue_dic_f[0],dialogue_dic_f[1],dialogue_dic_f[2],50)
    for fx in er_result_f:
        fx['file'] = f
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)

In [147]:
list1 = ['hello','love','life']
c = list1.count('l')
print(c)

0


[]


IndexError: list index out of range